'''

Predictive Patient Re-admission Model

Date: 14/03/18

Author: Annye Braca

Email: annyebraca@gmail.com

'''

In [ ]:
# Importing Libraries
from __future__ import absolute_import, division, print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
from scipy.stats import spearmanr
from pylab import rcParams
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn import feature_extraction
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import model_selection
#import statsmodels.api as sm
import  sys
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time
import keras
from keras.layers import Flatten, Conv1D, Dense, GlobalAvgPool1D, GlobalMaxPool1D, Dropout, MaxPool1D, BatchNormalization

In [ ]:
def one_hot_dataframe(data, cols, replace=False):
    """Create one-hot encodings."""
    vec = feature_extraction.DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData)

In [ ]:
# Read in data
data = pd.read_csv('../static/diabetic_data.csv')
discharge = pd.ExcelFile('../static/discharge.xlsx')
discharge = discharge.parse('Data')
admission = pd.ExcelFile('../static/admission.xlsx')
admission = admission.parse('Data')
admission_type = pd.ExcelFile('../static/admission_type_ID.xlsx')
admission_type = admission_type.parse('Data')

# Map discharge and admission id descriptions
discharge_map = dict(zip(discharge['discharge_disposition_id'], discharge['description']))
data['discharge_description'] = data['discharge_disposition_id'].map(discharge_map)
admission_map = dict(zip(admission['admission_source_id'], admission['description']))
data['admission_description'] = data['admission_source_id'].map(admission_map)
# bring in admission type id
admission_type_map = dict(zip(admission_type['admission_type_id'], admission['description']))
data['admission_type_description'] = data['admission_type_id'].map(admission_type_map)
# Drop original ids as these are not dropped in one-hot_encoding function
data.drop(['discharge_disposition_id', 'admission_source_id', 'admission_type_id'], axis=1, inplace=True)

# Mapping Age
age_ranges = list(set(data['age']))#
age_ordinal = [1, 8, 7, 6, 0, 9, 4, 2, 5, 3]
age_map = dict(zip(age_ranges, age_ordinal))
data['age'] = data['age'].map(age_map)


data.replace('?' , np.nan, inplace=True)
data.drop(['weight', 'medical_specialty', 'payer_code', 'diag_2', 'diag_3',  'patient_nbr', 'encounter_id'], axis=1, inplace=True)

data = data[~(data['diag_1'].str.startswith('V') | (data['diag_1'].str.startswith('E')))]


data.dropna(inplace=True)

In [ ]:
data, med = one_hot_dataframe(data, ["admission_description",
                                     "discharge_description",
                                     "admission_type_description",
                                     "race",
                                     "gender",
                                     "metformin",
                                     "repaglinide",
                                     "nateglinide",
                                     "chlorpropamide",
                                     "glimepiride",
                                     "acetohexamide",
                                     "glipizide",
                                     "glyburide",
                                     "tolbutamide",
                                     "pioglitazone",
                                     "rosiglitazone",
                                     "acarbose",
                                     "miglitol",
                                     "troglitazone",
                                     "tolazamide",
                                     "examide",
                                     "citoglipton",
                                     "insulin",
                                     "glyburide-metformin",
                                     "glipizide-metformin",
                                     "glimepiride-pioglitazone",
                                     "metformin-rosiglitazone",
                                     "metformin-pioglitazone",
                                     'A1Cresult',
                                     'max_glu_serum',
                                     'change',
                                     'diabetesMed'], replace=True)


In [ ]:
inds = pd.isnull(data).any(1).nonzero()[0]
data.ix[inds]

In [ ]:
set(data['readmitted'])
data = data[data['diabetesMed=Yes'] == 1]
print ('Diabetes patients: {}'.format(len(data)))
print ('Readmitted: No: {}'.format(len(data[data['readmitted'] == 'NO'])))
print ('Readmitted: <30: {}'.format(len(data[data['readmitted'] == '<30'])))
print ('Readmitted: >30: {}'.format(len(data[data['readmitted'] == '>30'])))

Binary Mapping

In [ ]:
#mapping readmitted
readmitted_ranges = list(set(data['readmitted']))
readmitted_ordinal = [1, 1, 0]
readmitted_map = dict(zip(readmitted_ranges, readmitted_ordinal))
data['readmitted'] = data['readmitted'].map(readmitted_map)

In [ ]:
###########Subsetting data  , we are interested just in people with a Diabetes condition  #################

In [ ]:
diabetes = data[data['diabetesMed=Yes'] == 1]

In [ ]:
len(diabetes) 

In [ ]:
X = diabetes
inds = pd.isnull(X).any(1).nonzero()[0]
y = diabetes['readmitted']
X.drop(['readmitted', 'readmitted'], axis=1, inplace=True)

In [263]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.624


In [193]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.62      0.71      0.67      8857
          1       0.63      0.53      0.57      8136

avg / total       0.63      0.63      0.62     16993



In [ ]:
###############Convolutional neural networks####################

In [206]:
from keras.regularizers import L1L2

In [214]:
def create_baseline(input_nodes,
                    hidden1_nodes,
                    hidden2_nodes,
                    hidden3_nodes,
                    output_nodes,
                    dropout_rate):
        # NN architecturE
        model = Sequential()
        #model.add(GlobalAvgPool1D(input_shape=(146,2,)))
        model.add(Dense(output_dim=hidden1_nodes,
                        input_dim=input_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden2_nodes,
                        input_dim=hidden1_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden3_nodes,
                        input_dim=hidden2_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=hidden4_nodes,
                        input_dim=hidden3_nodes,
                        activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(output_dim=output_nodes,
                        input_dim=hidden4_nodes,
                        activation='sigmoid'))
        # Compile model
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        return model

In [200]:
# categorical_crossentrophy expects targets to be binary matrices - if target are integer classes
# we need to convert to the expected format using to categorical
# fix random seed for reproducibility
y_train = np.array(y_train)
y_train = [[x] for x in y_train]
y_train = np.array(y_train)
y_test = np.array(y_test)
y_test = [[x] for x in y_test]
y_test = np.array(y_test)

In [216]:
""" Keras Sequential model with Manual validation
"""
start_time = time.time()


# Only for categorical_crossentrophy - Comment out for binary_crossentrophy
#y_train = keras.utils.to_categorical(y_train, num_classes=3)
#y_test = keras.utils.to_categorical(y_test, num_classes=3)

seed = 13
np.random.seed(seed)
cvscores = []
learning_rate = 0.1
sgd = SGD(lr=learning_rate, momentum=0.05, nesterov=False)

input_nodes = 146
hidden1_nodes = 50
hidden2_nodes = 50
hidden3_nodes = 50
hidden4_nodes = 50
output_nodes = 1
dropout_rate = 0


model = create_baseline(input_nodes,
                        hidden1_nodes,
                        hidden2_nodes,
                        hidden3_nodes,
                        output_nodes,
                        dropout_rate)



model.fit(X_train, y_train, epochs=100, batch_size=120) 
print('--- %s seconds ---'%(time.time() - start_time))
'''
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model,
                            nb_epoch=100,
                            batch_size=200,
                            verbose=2)
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=seed)
results = cross_val_score(estimator,
                          X,
                          y,
                          cv=kfold)
'''
#print("Results of 10-fold Cross-Validation: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

C:\Users\Creative\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=146, units=50, activation="relu")`
  if sys.path[0] == '':
C:\Users\Creative\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, units=50, activation="relu")`
  app.launch_new_instance()
C:\Users\Creative\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, units=50, activation="relu")`
C:\Users\Creative\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=50, units=50, activation="relu")`
C:\Users\Creative\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_

Epoch 1/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.7292 - acc: 0.5424
Epoch 2/100
50977/50977 [==============================] - 5s 97us/step - loss: 0.6697 - acc: 0.5870
Epoch 3/100
50977/50977 [==============================] - 6s 116us/step - loss: 0.6579 - acc: 0.6029
Epoch 4/100
50977/50977 [==============================] - 5s 105us/step - loss: 0.6566 - acc: 0.6062
Epoch 5/100
50977/50977 [==============================] - 6s 112us/step - loss: 0.6535 - acc: 0.6082
Epoch 6/100
50977/50977 [==============================] - 6s 117us/step - loss: 0.6497 - acc: 0.6131
Epoch 7/100
50977/50977 [==============================] - 5s 107us/step - loss: 0.6478 - acc: 0.6150
Epoch 8/100
50977/50977 [==============================] - 5s 101us/step - loss: 0.6450 - acc: 0.6185
Epoch 9/100
50977/50977 [==============================] - 5s 103us/step - loss: 0.6436 - acc: 0.6200
Epoch 10/100
50977/50977 [==============================] - 5s 96us/step - loss: 0.

'\n# evaluate model with standardized dataset\nestimator = KerasClassifier(build_fn=model,\n                            nb_epoch=100,\n                            batch_size=200,\n                            verbose=2)\nkfold = StratifiedKFold(n_splits=10,\n                        shuffle=True,\n                        random_state=seed)\nresults = cross_val_score(estimator,\n                          X,\n                          y,\n                          cv=kfold)\n'

In [ ]:
# Separate training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

In [42]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(model.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.88


In [43]:

kfold = model_selection.KFold(n_splits=10, random_state=7)
modelCV = LogisticRegression()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.884


In [44]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.51      0.02      0.03      2388
          1       0.88      1.00      0.94     18003

avg / total       0.84      0.88      0.83     20391



In [215]:
# Separate training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)

In [90]:
# ANOVA SVM-C
# 1) anova filter, take 3 best ranked features
anova_filter = SelectKBest(f_regression, k=20)
# 2) svm
clf = svm.SVC(kernel='linear')

anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train)
y_pred = anova_svm.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.59      0.85      0.69     10559
          1       0.69      0.35      0.46      9832

avg / total       0.64      0.61      0.58     20391



In [221]:
model.coef_

array([[-5.93975818e-03,  1.42867668e-02,  7.45472352e-05,
        -3.51171599e-02,  2.81204559e-03,  6.29392147e-02,
         1.80889689e-01,  3.75142184e-01, -2.16043647e-04,
         7.58963518e-02, -2.43135645e-02, -6.15495609e-02,
         8.73396877e-02, -5.54292993e-02, -3.50026573e-01,
        -2.63251593e-01,  3.26598471e-01,  2.32726958e-01,
        -2.88199462e-01,  2.34246725e-01,  1.75253572e-02,
         2.34867131e-01, -2.13602966e-01,  7.53518102e-02,
         4.50047413e-01, -1.78655931e-01,  2.65301942e-01,
        -2.17205260e-01,  1.11889340e-02,  9.64887986e-01,
        -3.72217565e-01,  5.22496554e-02, -3.30529102e-01,
        -1.95218201e-01, -3.79125560e-01, -2.38818381e-01,
        -2.15184643e-01,  1.22063795e-02,  1.75192771e-01,
        -1.31680956e+00,  2.99852895e-02,  5.57717022e-01,
         3.23849911e-01,  3.79090094e-01, -5.10447175e-02,
        -2.90801947e-03, -2.18156177e-01, -2.07400106e-01,
        -1.36513266e-02,  3.85254872e-01, -5.39527370e-0